#Cloning YOLOv3 and DataSet

We cloned Ultralytics's yolov3 model and preformed transfer learning on the last three layers of yolov3.

The dataset we used for this project available for free on roboflow.
We added more augmentations making the dataset x3 times bigger
And changed the number of classes from 13 to 6 (detecting without color).

Total of: 1818 train images  |  174 validation images | image size 614x614



In [ ]:
!git clone https://github.com/ultralytics/yolov3.git

!pip install "opencv-python-headless<4.3"
! pip install --upgrade albumentations

!curl -L "PUT YOUR DATASET HERE FROM ROBOFLOW" | jar -x

Cloning into 'yolov3'...
remote: Enumerating objects: 9960, done.
remote: Total 9960 (delta 0), reused 0 (delta 0), pack-reused 9960
Receiving objects: 100% (9960/9960), 9.31 MiB | 29.34 MiB/s, done.
Resolving deltas: 100% (6718/6718), done.
     |████████████████████████████████| 21.6 MB 1.3 MB/s 
     |████████████████████████████████| 102 kB 6.7 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
%cd yolov3
!pip install -r requirements.txt  # install dependencies
%cd ..

/content/yolov3
     |████████████████████████████████| 596 kB 5.1 MB/s 
     |████████████████████████████████| 1.7 MB 43.5 MB/s 
     |████████████████████████████████| 180 kB 46.9 MB/s 


# Imports

In [ ]:
import os
import shutil
import cv2
import albumentations as A
from albumentations.augmentations.transforms import VerticalFlip
from albumentations.augmentations.geometric.rotate import Rotate

from matplotlib import pyplot as plt
import torch

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.10.0+cu111 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


# Augmantations
used Albumentation library.
1.	First augmentation: gaussianNoise + MotionBlur + Rotation.
2.	Second augmentation: HorizontalFlip + Verticalflip + Brightness + Rotation.
 
For each image augmentation we created a new txt file corresponding to the new image name (oldname + A/B + .jpg/.txt) containing the new bounding boxes.


In [ ]:


transform_rot1 = A.Compose([
        A.GaussNoise(),
        A.RandomGamma(),
        A.MotionBlur(),
        A.Rotate(p=0.9, border_mode=cv2.BORDER_CONSTANT),  
], bbox_params=A.BboxParams(format='yolo', label_fields=[]))

transform_rot2 = A.Compose([
    A.HorizontalFlip(p=0.6),
    A.VerticalFlip(p=0.2),
    A.Rotate(p=0.9, border_mode=cv2.BORDER_CONSTANT),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=[]))



In [ ]:

def makeAugmantations(x):
  %cd {x}

  for filename in os.listdir("."):
    if filename.endswith(".jpg"):
      #make list of bboxes: floats yolo format
      f = open(filename[:-4] + '.txt' , "r")
      cls1 = []
      cls2 = []
      boxes = []
      for line in f:
        stripped_line = line.strip()
        line_list = stripped_line.split()
        float_list = [float(i) for i in line_list]
        cls1.append(line_list[0])
        cls2.append(line_list[0])
        boxes.append(float_list[1:])
      f.close()
      imageORG = cv2.imread(filename)
      image = cv2.cvtColor(imageORG, cv2.COLOR_BGR2RGB)
      
      augmented1 =transform_rot1(image=image, bboxes=boxes)
      cv2.imwrite(filename[:-4] + "A.jpg", augmented1["image"])
      New_boxes =  augmented1["bboxes"]
      f = open(filename[:-4] + "A.txt", 'w')
      for line in New_boxes:
        f.write(str(cls1[0]) + ' ')
        del cls1[0]
        f.write(' '.join(str(s) for s in line) + '\n')
      f.close()
      
      augmented2 =transform_rot2(image=image, bboxes=boxes)
      cv2.imwrite(filename[:-4] + "B.jpg", augmented2["image"])
      New_boxes =  augmented2["bboxes"]
      f = open(filename[:-4] + "B.txt", 'w')
      for line in New_boxes:
        f.write(str(cls2[0]) + ' ')
        del cls2[0]
        f.write(' '.join(str(s) for s in line) + '\n')
      f.close()

      augmented3 = cv2.Canny(imageORG, 100, 200)

      #cv2.imwrite(filename[:-4] + "C.jpg", augmented3)
      #%cp {filename[:-4]}.txt {filename[:-4]}C.txt




In [ ]:
print(os.getcwd())
makeAugmantations("/content/train")
makeAugmantations("/content/valid")

    

/content
/content/train
/content/valid


# separate between txt ang jpg
for train and valid

1. for each jpg there is corresponding txt file with information about the label and bounding boxes of that jpg image. 
[label x y w h]    
(x,y,w,h are normalized).

2. Making mapping files: "images_train" and "images_valid"                   (for convenience we made it all organized into folders).                      Each file is a list paths for all the images on train/valid dataset


In [ ]:
%cd /content/train 
%mkdir labels
%mkdir images
%mv *jpg ./images/
%mv *txt ./labels/
%cd images
# create ultralytics specific text file of training images


file = open("images_train.txt", "w")
for root, dirs, files in os.walk("."):
    for filename in files:
      if filename == "images_train.txt":
        pass
      else:
        file.write("../train/images/" + filename + "\n")

file.close()
%cat images_train.txt

/content/train
/content/train/images
../train/images/d079f4e77b2445abceca7534356db743_jpg.rf.15980e0041c274dd182fc8a6bdd25047B.jpg
../train/images/4eb630d4dd38528dacf72355caf5c06d_jpg.rf.d291d9ff254a5001b29ef8ce2d722509A.jpg
../train/images/fa4e2b9a8cf58f405f69a56c662834f2_jpg.rf.29021f89f2a16c44ecb9f34623bd3a9aB.jpg
../train/images/e8480d7fb9881d8a0e88b7be4d103f6d_jpg.rf.f5a8297fdaadf713f5aa50520a5aae2fA.jpg
../train/images/3474d785b1b21d68163f56aa00a92bc9_jpg.rf.526eb41b7d4b9055bcd36adc5d65734aA.jpg
../train/images/9e943906fba1ec89edfacb2dd7976504_jpg.rf.6e9e4b1b2d5fb537ddde22c5fbbff591B.jpg
../train/images/0d9dbf62d5ee42b92bf55197bba4254d_jpg.rf.214eb5f360891da5197e1900013439ecB.jpg
../train/images/IMG_0298_JPG.rf.6d6ac51367b303c05c261e7e8b11de67.jpg
../train/images/ce54969567273b9b8a275812ff56e16c_jpg.rf.9dcc1b3107606542dc0633cbc7ff5026.jpg
../train/images/79e744a68d6e6f83be0a9e8761ea66a4_jpg.rf.a7b671ff5bbf39d29460db7e874fe65e.jpg
../train/images/cae099fe41d6aa30033d71e433c33c8d_j

In [ ]:
# create ultralytics specific text file of training images
%cd /content/valid/
%mkdir labels
%mkdir images
%mv *.jpg ./images/
%mv *.txt ./labels/
%cd images


file = open("images_valid.txt", "w")
for root, dirs, files in os.walk("."):
    for filename in files:
      if filename == "images_valid.txt":
        pass
      else:
        file.write("../valid/images/" + filename + "\n")

file.close()
%cat images_valid.txt


/content/valid
/content/valid/images
../valid/images/4b8f93069270a9f7bb523518a5088b9e_jpg.rf.7d0d7a337008fbfde0ab2c55715e7782B.jpg
../valid/images/abd65798d9952a27e087710eb8bddf32_jpg.rf.4c9ed4ea8f55c6b3b1b007060029a15f.jpg
../valid/images/26fa37995fa5b18ec40e0a94e6d91104_jpg.rf.2e6b063142cfe7cbcc21b0a236e5699f.jpg
../valid/images/IMG_0293_JPG.rf.e208f5cdf5e993c552be7f96e86c4890B.jpg
../valid/images/d9ef98145d7d35393c75a51331a20e2c_jpg.rf.44441521d89767a579e4c3f3ab44d4a8B.jpg
../valid/images/aec1aa6773dbbe004554f405cdef2bea_jpg.rf.2775fd3d7d8d1dfacea70de121d91437A.jpg
../valid/images/e53bf8a0e692a4ccd5f1dc2bc19e7751_jpg.rf.a8b866893eddbe54b209aa055984e2e6.jpg
../valid/images/IMG_0310_JPG.rf.fa4dfa05ea9b136a2afc53556d90adb9A.jpg
../valid/images/IMG_0293_JPG.rf.e208f5cdf5e993c552be7f96e86c4890A.jpg
../valid/images/d3b9309d00a2b671407b918ea867a935_jpg.rf.5e1400d76ea181788b7defd2f5d64254.jpg
../valid/images/3baf85c957b9d28a16c0b65cb2ef0d29_jpg.rf.712429bc0be682ad0038cbc6beb3fd30B.jpg
../va

**Set up model config**

COCO have 80 classes and we will add the chess pieces classes

In [ ]:
%cd /content/yolov3/data

#convert .labes to .name for ultralytics
%cat ../../train/_darknet.labels > ../../train/data.names

/content/yolov3/data


# Train - transfer learnig
By running 'train.py' we will train our model.

Train.py accepts argument… the one we used:

*   " --data chess_yolov3.yaml " gives the model the new metadata configurations such as number of classes and essential paths.
*   "--weights yolov3.pt" initiate the model with pretrained weights. This specific weights were trained on COCO dataset with 80 different classes.
*   " --epochs 50" : number of epochs.
*   "--img 614" : image input shape. updated to 640
*   "--freeze 24" : The tranfer-learning part.



The yolov3 architecture of 106 layers is divided to 28 sublayers (wrap). 
*  	0-9 layers are the backbone darknet network.
*  	10 – 27 layers are the additional yolov3 layers.
* 	28 - The last layer is the final detection layer.
For transfer learning we will try different freezing values:
1.	10  – run very slow and crushes due to space allocation errors.
2.	24 – runs faster, don’t make any errors, training last detection layer.
3.	28 – fastest, train only the the last output detection layer.



In [ ]:

%cd /content/yolov3
!python3 train.py --data chess_yolov3.yaml --weights yolov3.pt --freeze 24  --epochs 50  --batch-size 1

/content/yolov3
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 1
wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
train: weights=yolov3.pt, cfg=, data=chess_yolov3.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=24, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest


# Detection

For detection we will use a home made video of us playing chess.
The detection takes please at "detect.py" and we pass the following arguments:
1.	"--weights best_weight"  - the weight with the best results from all our runs.
2.	"--conf 0.25" : detection box threshold, changed the number for better precision.
3.	"--source input.mp4 " – input video to detect on.
The output detections saved in yolov3/rund/detect/… in the same format given.
4.  "--line-thickness 1" - line thickness in pixels.
5.  "--max-det 14" - max detections per image.
6.  "--hide-conf" - hide confidence percentage.

In [ ]:
%cd /content/yolov3
!python detect.py --weights ../drive/MyDrive/chess_AUG/nodel_24_6cls/weights/best.pt --line-thickness 1 --hide-conf --max-det 14  --source ../444.mp4



[Errno 2] No such file or directory: '/content/yolov3'
/content
python3: can't open file 'detect.py': [Errno 2] No such file or directory
